# Installation

Apart from the ambient music and video, the game's files are contained in InstallShield cabinet files. I actually did a *lot* of reverse engineering of these before finding [Unshield](https://github.com/twogood/unshield). Ultimately, this is not a good approach. It's much easier to install the game, after which the files are unpacked onto the hard drive!

I used [VirtualBox](https://www.virtualbox.org/) (free, Windows/macOS/Linux) to install the game in a virtual machine (VM). This could be modern Windows VM, such as [the free Windows 10 VM](https://developer.microsoft.com/en-us/windows/downloads/virtual-machines) offered by Microsoft, or [an old Windows 95 VM](https://archive.org/details/windows_95_vdi) of [archive.org](https://archive.org/).

I used the Windows 95 VM, for "authenticity". This presents unique challenges, such as getting files onto and off the VM, because networking does not work \*. For transferring files onto the VM, I created an `.iso` disk image containing the patches, which I could then mount into the VM using the virtual CD drive. For transferring files off the VM, I shut it down and used [7-zip](https://www.7-zip.org/) to copy the files out of the virtual disk image.

Seriously though, use a modern Windows VM. Yes, modern Windows OS' have compatibility issues running the game. But the important part is installing (and maybe patching the game), not running it, so that'll do.

---

\* I'm sure someone with more 95 and VirtualBox experience *could* get it to work...

Time to boot up the glorious installer! God damn, that Mad Cat.

![The initial screen of the install "wizard"](screenshot/installer-01.png)

EULAs remain an abomination to this day (shrink-wrapped EULAs are void in Germany).

!["Please confirm that you accept the terms of this agreement"](screenshot/installer-03.png)

By default, the game installs to `C:\MicroProse\Mechwarrior3`.

!["Please enter the installation location"](screenshot/installer-04.png)

Now, you can chose which components to install. A compact install is 216 MB, a typical install is 404 MB, although this can be more with video files also installed.

![Select Components](screenshot/installer-09.png)

Here is an overview of components with sub-choices, depending on hardware:
```
Software Render Files
	Low Detail
	Medium Detail (not checked by default)
	Best Detail
3D Accelerator Files
	2 MB Card
	4 MB Card
	8 MB Card+
Sound
	Low Fidelity (not checked by default)
	High Fidelity
```

For reverse engineering, it's best to install all of them so there's more files to examine. If you want to preserve the best version of MechWarrior 3 assets, then only "3D Accelerator Files - 8 MB Card+" and "Sound - High Fidelity" are required.

## Patching

In any case, you'll probably want to install the 1.2 patch, for example from [sarna.net](http://www.sarna.net/files/programs/games/mechwarrior_3/patches/). I couldn't find a working 1.1 patch, but that would mainly be for comparisons and completeness.

## Game files

The top-level installation folder is rather bare, as all the game files are all in the `zbd` folder. Inside that, there are some files ending in `*.zbd`. As we will see later, this is not one file type, but just a naming convention.

Let's have a guess what these files could be:

* `mechlib.zbd` is the one that jumps out first to me, and I hope it contains the 'mech models
* `interp.zbd` is probably short for interpolation, and could be something to do with 'mech animations...
* `motion.zbd` ...but then so could this file. So the previous one may be short for "interpreter", related to game functionality or campaigns
* `reader.zbd` seems to be generic program data, as there are more of them in sub-folders
* `rimage.zbd` is no doubt image assets
* `rmechtex.zbd`, `rmechtex16.zbd`, `rmechtexs.zbd` should be textures for 'mech models
* `soundsH.zbd`, `soundsL.zbd` these are the high and low sound effects collections, regardless.

There are also some sub-folders, `c1`, `c2`, `c3`, `c4`, `c4b`, and `t1`. Venturing another guess, this is mission data. `t1` contains assets for the training missions, and the others are <b>c</b>ampaigns or <b>c</b>hapters (called operations in-game). One oddity is `c4b`, which is possibly because the third and fourth operations had 6 missions each (instead of four), and there was some kind of game engine limitation. Clever!


Here's a matrix of the installation options the files correspond to, with some files being used in multiple configurations:

| Render   | Low | Medium | High |
|----------|-----|--------|------|
| Software | `texture1.zbd` | `texture2.zbd` | `texture.zbd` |
| Hardware | `rtexture2.zbd` | `rtexture3.zbd` | `rtexture.zbd` |
| Software | `rmechtexs.zbd` | `rmechtexs.zbd` | `rmechtexs.zbd` |
| Hardware | `rmechtex16.zbd` | `rmechtex.zbd` | `rmechtex.zbd` |


## Next up

Skip over the version comparison to [extracting sound effects](05-sounds.ipynb)

## Version comparison

As a reminder, I have US 1.0, US 1.1, US Gold Edition (ostensibly 1.2), and German (DE) 1.0. These are designated "pre" for pre-patched. I then patched all of them with 1.2. These files are designated "post".

```console
$ ls -l install/
total 192
drwxr-x--- v1.0-de-post
-rw-r----- v1.0-de-post.sha256
drwxr-x--- v1.0-de-pre
-rw-r----- v1.0-de-pre.sha256
drwxr-x--- v1.0-us-post
-rw-r----- v1.0-us-post.sha256
drwxr-x--- v1.0-us-pre
-rw-r----- v1.0-us-pre.sha256
drwxr-x--- v1.1-us-post
-rw-r----- v1.1-us-post.sha256
drwxr-x--- v1.1-us-pre
-rw-r----- v1.1-us-pre.sha256
drwxr-x--- v1.2-us-post
-rw-r----- v1.2-us-post.sha256
drwxr-x--- v1.2-us-pre
-rw-r----- v1.2-us-pre.sha256
```

I've also hashed all the files using SHA-256. This is good for archival, but we can also use the hashes to compare files across versions. Work harder, not smarter. If I can do less work, I will.

In [2]:
from pathlib import Path
from collections import defaultdict

install = Path("install")
files = defaultdict(dict)

# get all game files, and their corresponding hashes in the different versions
for sha_index in install.glob("*.sha256"):
    version = sha_index.stem
    with sha_index.open("r", encoding="utf-8") as f:
        for line in f:
            sha_hash, filename = line.split(maxsplit=1)
            # only want game files
            if not (filename.startswith("./zbd/") and filename.endswith(".zbd\n")):
                continue
            # strip "./zbd/" and newline
            filename = filename[6:-1]
            files[filename][version] = sha_hash


def compare_files(filename):
    hashes = files[filename]

    # make sure "pre" appears before "post"
    versions = sorted(
        hashes.keys(), key=lambda s: s.replace("pre", "a").replace("post", "b")
    )
    print(filename, end="\t")
    print(*range(1, len(versions) + 1), sep=" ")
    for i, v1 in enumerate(versions, 1):
        print(i, v1, end="\t")
        for v2 in versions:
            print("\033[32mm" if hashes[v1] == hashes[v2] else "\033[31mx", end=" ")
        print("\033[0m")
    print("---")

Below is the dump. I'll summarise it here. `mechlib.zbd`, `motion.zbd`, `rmechtex16.zbd`, and `rmechtexs.zbd` are the same for all versions. This is great news! `rimage.zbd` is the same for all US versions. This probably means there is some localised image in it. `rmechtex.zbd` is super interesting; it seems unaffected by any patch, but is different for the Gold Edition. The two sound archives `soundsL.zbd` and `soundsH.zbd` are also unaffected by the patch - this is because the patch simply writes `.wav` files into the directory instead of patching. Otherwise, they are different for each edition, but the simplistic patch behaviour makes them prime targets for early analysis. Finally, `interp.zbd` is all over the place, as is `reader.zbd`. Let's leave these for last :)

In [3]:
for filename in (
    "interp.zbd",
    "mechlib.zbd",
    "motion.zbd",
    "reader.zbd",
    "rimage.zbd",
    "rmechtex.zbd",
    "rmechtex16.zbd",
    "rmechtexs.zbd",
    "soundsH.zbd",
    "soundsL.zbd",
):
    compare_files(filename)

interp.zbd	1 2 3 4 5 6 7 8
1 v1.0-de-pre	m m x x x x x x 
2 v1.0-de-post	m m x x x x x x 
3 v1.0-us-pre	x x m x m x x x 
4 v1.0-us-post	x x x m x m x m 
5 v1.1-us-pre	x x m x m x x x 
6 v1.1-us-post	x x x m x m x m 
7 v1.2-us-pre	x x x x x x m x 
8 v1.2-us-post	x x x m x m x m 
---
mechlib.zbd	1 2 3 4 5 6 7 8
1 v1.0-de-pre	m m m m m m m m 
2 v1.0-de-post	m m m m m m m m 
3 v1.0-us-pre	m m m m m m m m 
4 v1.0-us-post	m m m m m m m m 
5 v1.1-us-pre	m m m m m m m m 
6 v1.1-us-post	m m m m m m m m 
7 v1.2-us-pre	m m m m m m m m 
8 v1.2-us-post	m m m m m m m m 
---
motion.zbd	1 2 3 4 5 6 7 8
1 v1.0-de-pre	m m m m m m m m 
2 v1.0-de-post	m m m m m m m m 
3 v1.0-us-pre	m m m m m m m m 
4 v1.0-us-post	m m m m m m m m 
5 v1.1-us-pre	m m m m m m m m 
6 v1.1-us-post	m m m m m m m m 
7 v1.2-us-pre	m m m m m m m m 
8 v1.2-us-post	m m m m m m m m 
---
reader.zbd	1 2 3 4 5 6 7 8
1 v1.0-de-pre	m x x x x x x x 
2 v1.0-de-post	x m x x x x x x 
3 v1.0-us-pre	x x m x x x x x 
4 v1.0-us-post	x x x m x m m 

In [5]:
for filename in (
    "c1/anim.zbd",
    "c1/gamez.zbd",
    "c2/anim.zbd",
    "c2/gamez.zbd",
    "c3/anim.zbd",
    "c3/gamez.zbd",
):
    compare_files(filename)

c1/anim.zbd	1 2 3 4 5 6 7 8
1 v1.0-de-pre	m m x x x x x x 
2 v1.0-de-post	m m x x x x x x 
3 v1.0-us-pre	x x m m m m x x 
4 v1.0-us-post	x x m m m m x x 
5 v1.1-us-pre	x x m m m m x x 
6 v1.1-us-post	x x m m m m x x 
7 v1.2-us-pre	x x x x x x m m 
8 v1.2-us-post	x x x x x x m m 
---
c1/gamez.zbd	1 2 3 4 5 6 7 8
1 v1.0-de-pre	m m x x x x x x 
2 v1.0-de-post	m m x x x x x x 
3 v1.0-us-pre	x x m m m m x x 
4 v1.0-us-post	x x m m m m x x 
5 v1.1-us-pre	x x m m m m x x 
6 v1.1-us-post	x x m m m m x x 
7 v1.2-us-pre	x x x x x x m m 
8 v1.2-us-post	x x x x x x m m 
---
c2/anim.zbd	1 2 3 4 5 6 7 8
1 v1.0-de-pre	m m x x x x x x 
2 v1.0-de-post	m m x x x x x x 
3 v1.0-us-pre	x x m m m m x x 
4 v1.0-us-post	x x m m m m x x 
5 v1.1-us-pre	x x m m m m x x 
6 v1.1-us-post	x x m m m m x x 
7 v1.2-us-pre	x x x x x x m m 
8 v1.2-us-post	x x x x x x m m 
---
c2/gamez.zbd	1 2 3 4 5 6 7 8
1 v1.0-de-pre	m m x x x x x x 
2 v1.0-de-post	m m x x x x x x 
3 v1.0-us-pre	x x m m m m x x 
4 v1.0-us-post	x x m m m

## Next up

[Extracting sound effects](05-sounds.ipynb)